In [1]:
import numpy as np
import pandas as pd

%pip install -Uqq fastai
from fastai.vision.all import *

from PIL import Image


Note: you may need to restart the kernel to use updated packages.


In [7]:

import wandb

wandb.init(

    project="image-segmentation",

    config={
    "learning_rate": 0.02,
    "architecture": "U-Net",
    "dataset": "CITYSCAPES",
    "epochs": 3,
    }
)

# def hook_fn_first(m, i, o):
#     feature_map_first = o[0]
#     wandb.log({"First Layer Feature Map": wandb.Histogram(feature_map_first)})

# def hook_fn_last(m, i, o):
#     feature_map_last = o[0]
#     wandb.log({"Last Layer Feature Map": wandb.Histogram(feature_map_last)})

path = Path(r'C:\Users\caiay\Downloads\archive\cityscapes_data')
sample_pct = 0.2
batch_size = 4

path_train = path/'train'
path_test = path/'val'

train_files = get_image_files(path_train)
test_files = get_image_files(path_test)

train_subset = random.sample(train_files, int(len(train_files) * sample_pct))
test_subset = random.sample(test_files, int(len(test_files) * sample_pct))

def split_input_label(ting):
    img = Image.open(ting)
    width = img.width // 2
    input_img = img.crop((0, 0, width, img.height))
    label_img = img.crop((width, 0, img.width, img.height))

    return input_img, label_img

dblock = DataBlock(blocks=(ImageBlock, ImageBlock),
                   get_items=get_image_files,
                   get_y=split_input_label,
                   splitter=RandomSplitter(valid_pct=0.2),
                   batch_tfms=[*aug_transforms()],
                   )

dls = dblock.dataloaders(path, train=train_subset, bs=batch_size)

# dls = SegmentationDataLoaders.from_folder(
#     path,
#     train=train_subset,
#     valid_pct=0.2,
#     test=test_subset,
#     batch_tfms=Normalize.from_stats(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
#     bs=batch_size)
loss_func = CrossEntropyLossFlat(axis=1) 
#DiceLoss()


model = unet_learner(dls, resnet18, n_out=30, loss_func=loss_func, metrics=[accuracy])

# hook_first = model.model[0][0].register_forward_hook(hook_fn_first)
# hook_last = model.model[-1].register_forward_hook(hook_fn_last)

# try:
model.fit(4)
# finally:
#     hook_first.remove()
#     hook_last.remove()

wandb.finish()


c:\Users\caiay\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\caiay\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,accuracy,time


AttributeError: 'tuple' object has no attribute 'dtype'